In [ ]:
import os
import warnings

import numpy as np

warnings.filterwarnings('ignore')
np.random.seed(123)

在本教程中，我们使用PetFinder 数据集的简化和二次采样版本。目标是根据宠物的收养情况来预测宠物的收养率。在此简化版本中，采用速度分为两类：0（慢）和 1（快）。我们首先下载包含 petfinder 数据集的 zip 文件并将其解压缩到当前工作目录中。

In [ ]:
from autogluon.core.utils.loaders import load_zip

download_dir = './ag_multimodal_tutorial'
zip_file = 'https://automl-mm-bench.s3.amazonaws.com/petfinder_for_tutorial.zip'

load_zip.unzip(zip_file, unzip_dir=download_dir)

接下来，我们使用 pandas 将数据集的 CSV 文件读入DataFrames，注意到我们有兴趣学习预测的列是“AdoptionSpeed”。

In [ ]:
import pandas as pd

dataset_path = f'{download_dir}/petfinder_for_tutorial'

train_data = pd.read_csv(f'{dataset_path}/train.csv', index_col=0)
test_data = pd.read_csv(f'{dataset_path}/test.csv', index_col=0)

label_col = 'AdoptionSpeed'

PetFinder 数据集附带一个图像目录，数据中的某些记录具有与其关联的多个图像。AutoGluon 的多模式数据帧格式要求图像列包含一个字符串，其值为单个图像文件的路径。对于此示例，我们将图像特征列限制为仅第一张图像，并且需要执行一些路径操作才能为当前目录结构正确设置所有内容。

In [ ]:
image_col = 'Images'

train_data[image_col] = train_data[image_col].apply(lambda ele: ele.split(';')[0])
test_data[image_col] = test_data[image_col].apply(lambda ele: ele.split(';')[0])

def path_expander(path, base_folder):
    path_l = path.split(';')
    return ';'.join([os.path.abspath(os.path.join(base_folder, path)) for path in path_l])

train_data[image_col] = train_data[image_col].apply(lambda ele: path_expander(ele, base_folder=dataset_path))
test_data[image_col] = test_data[image_col].apply(lambda ele: path_expander(ele, base_folder=dataset_path))

每只动物的收养资料包括图片、文字描述和各种表格特征，如年龄、品种、名称、颜色等。让我们看一下示例数据行的图片和说明。

In [ ]:
example_row = train_data.iloc[0]
example_image = example_row[image_col]

from IPython.display import Image, display
pil_img = Image(filename=example_image)
display(pil_img)

example_row['Description']

现在数据的格式合适，我们可以拟合MultiModalPredictor训练数据。在这里，我们为这个快速演示设置了严格的培训时间预算。更多的训练时间会带来更好的预测性能，但我们可以在短时间内获得令人惊讶的良好性能。

In [ ]:
from autogluon.multimodal import MultiModalPredictor

predictor = MultiModalPredictor(label=label_col).fit(
    train_data=train_data,
    time_limit=120
)

拟合模型后，我们希望用它来预测测试数据集中的标签。

In [ ]:
predictions = predictor.predict(test_data.drop(columns=label_col))
predictions[:5]

对于分类任务，我们可以轻松获得每个输出类别的预测概率。

In [ ]:
probs = predictor.predict_proba(test_data.drop(columns=label_col))
probs[:5]

最后，我们可以根据其他性能指标评估 witheld 测试数据集上的预测器，在本例中为roc_auc。

In [ ]:
scores = predictor.evaluate(test_data, metrics=["roc_auc"])
scores